In [1]:
import random
import sys
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics.cluster import adjusted_rand_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scripts.data_generator as data_generator
import scripts.feature_ranking as feature_ranking
import scripts.features_2d as features_2d
import scripts.ga as ga
import scripts.preprocess as preprocess
import scripts.ga_evaluation as ga_evaluation
import scripts.bio_analysis as bio_analysis
import tensorflow as tf
from IPython import get_ipython
from keras.backend.tensorflow_backend import set_session
from tqdm import tqdm
from collections import Counter
import seaborn as sns
import time

config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default

plt.ion()
plt.show()
sys.path.append("..")

random_state=1
random.seed( random_state )
np.random.seed(random_state)
method = "adapted_silhouette"

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5



In [2]:
%load_ext autoreload
%autoreload 2

# Steps
- Clustering 1d to select best discriminant features

- Clustering 2d to select redundant, close and outlier features

In [3]:
random_state=0
random.seed( random_state )
np.random.seed(random_state)
results = None

In [4]:
filenames = np.array(["khan","west",  "gordon",'borovecki'])
clustering = "hdbscan"
path = 'data/microarray/'
imp_f = np.arange(20)
for name in filenames:
    t1 = time.time()
    data = pd.read_pickle(f'{path}' + name + '.pkl')
    truth = data["truth"].values
    data = data.drop("truth", axis = 1).values


    n_clusters = len(np.unique(truth))

    z_file= f"data/microarray/Z_{name}_correlation.npy"
    print(f"\n##########  {name}, {data.shape}")

    # Clustering 1D
    meta_features = feature_ranking.rank_features(data,
                                              nb_bins=20,
                                              rank_threshold=90,
                                              z_file=z_file,
                                              metric='correlation',
                                              redundant_threshold=0.4)
    t2 = time.time()
    print(f"TIME: 1d Features : {(t2-t1)/60} min")
    model_file = f'models/gmm_arl.h5' 

    population, n = features_2d.run(data,
                                n_clusters,
                                meta_features,
                                model_file=model_file,
                                add_close_population=False)
    t3 = time.time()
    t4 = time.time()
    print(f"TIME: 2d scores: {(t4-t3)/60} min")
    round_size = 4
    epochs = 10*round_size
    sampling = {
        "ARCHIVE2D": { 
            "ga": 0.25,
            "max": 0.25 },
        "CLOSE": { 
            "ga": 0.4,
            "max": 0.4 },
        "IMP1D": { 
            "ga": 0.25,
            "max": 0.25 },
        "RANDOM": { 
            "ga": 0.1,
            "max": 0.1},
        }
    params = ga.ga_parameters(
        n_clusters,
        data.shape[1],
        truth,
        meta_features,
        method=method,
        truth_methods=['ari'],
        archive_2d=population[:data.shape[1] // 4],
        epochs=epochs,
        sampling = sampling,
        round_size=round_size,
        allow_subspace_overlap = True,
        improvement_per_mutation_report = False,
        clustering = clustering,
        total_maximisation_exploration = 300,
        max_subspace_size = 100,
        pca = False,
        hdbscan_min_cluster_size =4
        
    )
    solutions, archive= ga.run(data, params)
#     display(params["report"].groupby(["op", "improvement"]).count())
    solutions["dataset_name"] = name
    
    t5 = time.time()
    n_total = t5-t1
    print(f"TIME: GA: {(t5-t4)/60} min")
    print(f"TIME: Total: {(t5-t1)/60} min")
    solutions["total_time"] = round((t5-t1)/60, 1)
    solutions["t(feature_sel)"] = round((t2-t1)/60, 1)
    solutions["t(cnn)"] = round((t3-t2)/60, 1)
    solutions["t(clustering2d)"] = round((t4-t3)/60, 1)
    solutions["t(ga)"] = round((t5-t4)/60, 1)
    
    solutions["n_ga"] = archive.shape[0]
    solutions["n_cnn"] = n_total
    solutions["input_size"] = data.shape[1]
    
    
    solutions["n_non_redundant"] = meta_features[(meta_features["redundant"] ==1) ].shape[0]
    solutions["n_imp"] = meta_features[(meta_features["relevance"] !=0)].shape[0]
    solutions["n_imp4"] = meta_features[(meta_features["relevance"] ==4)].shape[0]
    solutions["n_imp3"] = meta_features[(meta_features["relevance"] ==3)].shape[0]
    
    
    if results is None: 
        results = solutions
    else:
        results = pd.concat([results, solutions], ignore_index = True)
    results.to_excel("reports/microarray_hdbscan.xlsx")
    


##########  khan, (63, 2308)
*** Computing 1D feature ranking ...
Dispersion tests took 0.03 sec
Entropy computation 0.49 sec
KNN computation 0.33 sec
Sorting and thresholds 0.04 sec
Loading clustering from file
Hierarchical clustering 0.02 sec
Handle redundant features 1.18 sec
Returning 381 redundant features and  83 important features
TIME: 1d Features : 0.03514524300893148 min
*** Exploring 2D feature space with NN ...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


trimming 1665 to 963
handle_close_important (963, 3), total 1678, 0.16005289554595947
relevant_features 606 => computing 3586 
trimming 3123 to 381
handle_important_features (1344, 3),  total 3586, 0.16005289554595947
irrelevant_features 1321 => computing 3953
trimming 3042 to 963
handle_not_important_features (2307, 3), total 3953, 0.16005289554595947
handle_all_features 1927 => computing 5772


  0%|          | 0/41 [00:00<?, ?it/s]

trimming 4666 to 1927
handle_all_features (4234, 3),  total 5772, 0.16005289554595947
Returning (4234, 3), explored a total of 14989 feature pairs
TIME: 2d scores: 3.178914388020833e-08 min
*** Optimization algorithm 
Non redundant features 1927, orig size 2308, nb imp : 83
Selecting (50, 4) from archive


 10%|▉         | 4/41 [00:09<01:35,  2.59s/it]

,adapted_silhouette,ari,features,partition,size,structure
0,0.615803,-0.01,"[35, 37, 112, 402, 406, 745, 854, 939, 954, 99...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, ...",34.0,"Counter({1: 42, 0: 21})"


...

 98%|█████████▊| 40/41 [04:26<00:06,  6.16s/it]

,adapted_silhouette,ari,features,partition,size,structure
0,0.605025,-0.03,"[231, 584, 960, 1083, 2960, 3175, 3325, 3431, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...",63.0,"Counter({0: 22, 1: 9})"
1,0.507162,0.03,"[5856, 5928, 14430, 19111, 20118]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",5.0,"Counter({0: 25, 1: 6})"
2,0.497364,-0.00,"[4297, 8523, 8814, 9834, 19904]","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, -1, -1, 0...",5.0,"Counter({1: 18, 0: 8, -1: 5})"
3,0.572232,0.54,"[1306, 2223, 2697, 2965, 2985, 3119, 3126, 324...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...",118.0,"Counter({1: 18, 0: 13})"
4,0.485984,-0.01,"[724, 1083, 1139, 1274, 2000, 2318, 2533, 2853...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, -1,...",60.0,"Counter({1: 20, 0: 7, -1: 4})"
5,0.469901,0.03,"[1444, 6171, 12729, 21347]","[-1, 1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 0, 1...",4.0,"Counter({1: 19, -1: 7, 0: 5})"
6,0.444249,0.00,"[6894, 9299]","[1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, ...",2.0,"Counter({1: 20, 0: 7, -1: 4})"
7,0.489161,-0.00,"[1804, 9434, 11073]","[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 0, 0, 1, 0,...",3.0,"Counter({1: 18, 0: 7, -1: 6})"
8,0.433967,0.00,"[1748, 2127, 9429, 18597]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...",4.0,"Counter({1: 26, 0: 5})"
9,0.521614,0.10,"[2417, 5760, 8070, 8090, 19336]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0,...",5.0,"Counter({0: 22, 1: 5, -1: 4})"


Selecting (3, 4) from archive


100%|██████████| 41/41 [04:39<00:00,  6.83s/it]


TIME: GA: 4.666820232073466 min
TIME: Total: 6.510584286848704 min


In [5]:
results.groupby("dataset_name").agg({"ari": max})

,ari
dataset_name,
borovecki,0.54
gordon,0.74
khan,0.41
west,0.10
